# Identifying Pneumonia with Deep Learning
***
Author: Andre Layton
***

# Overview
Artificial Intelligence has grown at a rapid rate in recent years, and has begun to creep into a myriad of industries for various uses. One such use is in medical imaging. Classification tasks are common in medical image analysis; it involves classifying medical images, such as X-rays, MRI scans, and CT scans, into different categories based on factors, like the type of image or the presence of specific structures/diseases. The aim is to use algorithms to classify medical images for assistance in diagnosis, treatment planning, and disease monitoring. The issue becomes how to correctly, or accurately, classify images - which is where deep learning models and neural networks come in handy.

This analysis will examine chest X-ray images of both healthy (referred to as "normal") and pneumonia-ridden patients in order to create a model that will accurately classify the two groups. The dataset comes from Kermany, Goldbaum et al.; and it is also available via Kaggle. The dataset is pre-split into training, testing, and validation directories, which makes loading and manipulating the data simple for my study. The data is also prepared in a way that makes it perfect for deep learning analysis (i.e. structured data with dimensions that don't require heavy processing). There are 5,216 images in the training directory (split into "NORMAL" and "PNEUMONIA" sub-directories), as well as 624 testing images and 16 validation images, both of which are split similarly to the training directory. I increased my validation dataset by splitting the testing data in order to train the model more effectively - this would have been tough with only 16 images. 

Once the data was loaded, I reshaped and standardized the training and validation images to prepare for modeling. I also reshaped their corresponding labels. Failing to standardize the dataset could have lead to skewed results, considering the wide range of the pixel values in the training data (e.g. \[0, 255] vs. \[0, 1] after standardizing). 

I used Keras, an effective high-level neural network (API), and TensorFlow, an end-to-end open-source deep learning framework that we run Keras on top of, in order to build my algorithms and determine the best model for image classification. I also used Scikit-learn, specifically the `KFold` validator, to cross-validate the baseline model, in order to determine a baseline accuracy level. Other modeling libraries used include `RMSNet50`, for transfer learning; `Conv2D` and `MaxPooling2D`, for a convolutional neural network; `ImageDataGenerator`, for data augmentation; and `ModelCheckpoint` along with `EarlyStopping`, to help save the best model. 

I began by building a baseline model - a neural network with two hidden layers - with hyperparameters I selected from prior research (i.e. RMSprop as my optimizer and binary cross-entropy for my loss). The next step was to use L2-regularization in order to combat the overfitting the former model exhibited. After, I used KFold to cross-validate the training model, and established my baseline accuracy score. I built a Convolutional Neural Network (CNN) next due to its effectiveness in classifying images, then augmented the training data in order to create more training data. More data allows the model to combat the high variance, and thus reduce the gap between the training and validation accuracies. I ended my modeling workflow with a transfer learning approach, specifically using the `RMSNet50` CNN pre-trained network to identify the surface patterns, before freezing the base and building another model to identify the abstract features in the training data. This approach also allows for data augmentation, paving the way for further improvement. 

The augmented CNN model yielded the best validation accuracy, while also remaining close enough to the training data's score that it avoided obvious overfitting. Luckily, the model was saved using the ModelCheckpoint technique, which was then applied to the testing images for evaluation. The model yielded 82% accuracy after testing 524 testing images, and exhibited the smallest margin of loss; however, there is much room for improvement and further work.

# Business Problem
Physicians and imaging labs are always looking for assistance in diagnosing illnesses, in order to improve treatment planning and health monitoring. The business objective is to build an algorithm that will accurately classify X-ray images (specifically chest images in this project) between two classes ("NORMAL" and "PNEUMONIA"). I'll be using data, available on Kaggle, that was gathered by Kermany, Goldbaum et al.
> Note: It is vital you run the `'validation_data_preparation.ipynb'` file prior to beginning this project, if you or another attempts to mimic the work. The file contains data preparation steps that are mentioned below. 

***
<img src="https://media.licdn.com/dms/image/D4D12AQEqfniioS4KVQ/article-cover_image-shrink_720_1280/0/1677311223167?e=2147483647&v=beta&t=loMCjRjjqYyKI72mjdbVk5_XbxRdzonIKz0qRn-BcXo" style="height:500px; width:875px" />

## Data Understanding
To begin, I import all the relevant libraries, and set a seed to be able to reproduce the results from this analysis. Next, I create objects containing the pathway for each directory so that I can begin examining the data. I use the `ImageDataGenerator` function I imported to generate the image data in each directory, and then split the datasets into two groups - images (input) and labels (output). 
> Note: In a separate notebook (`'validation_data_preparation.ipynb'`), I increased the number of validation images from 16 to 116 images, by using the `os` and `shutil` modules to move random files from the testing directories. 


In [ ]:
# Import all the necessary libraries
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.utils import get_file
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.wrappers import scikit_learn
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers, optimizers
from keras.applications import ResNet50
import os, shutil, random

# Create a seed for reproducibility
seed=24

In [ ]:
# Create directory path objects
train_data_dir = 'data/chest_xray/train'
test_data_dir = 'data/chest_xray/test'
validation_dir = 'data/chest_xray/val'

# Gather and reshape the data in each directory object
train_generator = ImageDataGenerator().flow_from_directory(train_data_dir, target_size=(64, 64), batch_size=5216,
                                                           seed=seed)
test_generator = ImageDataGenerator().flow_from_directory(test_data_dir, target_size=(64, 64), batch_size=524,
                                                           seed=seed)
val_generator = ImageDataGenerator().flow_from_directory(validation_dir, target_size=(64, 64), batch_size=116,
                                                           seed=seed)

# Create the datasets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

As you can see above, there are 5,216 training images, 116 validation images (thanks to the initial data preparation), and 524 testing images. Now that the data has been loaded and split, I preview a couple of images as plots for a look at what I'll be working with.

In [ ]:
# Plot some images
plt.imshow(train_images[17]/255)
plt.show()

plt.imshow(train_images[4200]/255)
plt.show()

To get a better idea of my data, I also list the shape of both the images and labels for each dataset. As you can see, the images have four dimensions - the first is the number of images, the middle values represent the size of the images, and the fourth value represents the number of channels. 

In [ ]:
# Preview the shape of the data
print("Train Data Shape:")
print(np.shape(train_images))
print(np.shape(train_labels))
print("\nTest Data Shape:")
print(np.shape(test_images))
print(np.shape(test_labels))
print("\nValidation Data Shape:")
print(np.shape(val_images))
print(np.shape(val_labels))

I also list the classes I will be working with in this analysis before reshaping the labels. Reshaping the classes now is only meant to examine whether the images truely correspond with their labels. I confirm this with the two following image plots, where the first accurately displays a "normal", or healthy, image, and the second displays an image for a pneumonia-ridden patient. 

In [ ]:
# List the classes
train_generator.class_indices

In [ ]:
# Reshape the labels
y_train_labels = train_labels.T[[1]]
y_test_labels = test_labels.T[[1]]
y_val_labels = val_labels.T[[1]]

# Check the shape of the transformed labels 
print(np.shape(y_train_labels))
print(np.shape(y_test_labels))
print(np.shape(y_val_labels))

In [ ]:
# Confirm that the image matches with its corresponding label
plt.imshow(train_images[1400]/255)
plt.show()
print(y_train_labels[:, 1400])

In [ ]:
# Confirm that the image matches with its corresponding label
plt.imshow(train_images[14]/255)
plt.show()
print(y_train_labels[:, 14])

As one can see, not much data preparation was needed for data understanding. However, there is still a need to transform the data to prepare it for modeling, which is the next step.

## Modeling

### Baseline Model:
As I mentioned earlier, the training and validation images will need to be reshaped, along with their labels, so that they contain the same first dimension. In addition, I standardize the images by dividing by 255 (due to the range of pixel intensity values falling between \[0, 255]\).

In [ ]:
# Reshape the train, test, and validation images for modeling
X_train = train_images.reshape(5216, -1)
X_test = test_images.reshape(524, -1)
X_val = val_images.reshape(116, -1)

# Preview the shape of each newly-formed objects
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(X_val))

In [ ]:
# Standardize the datasets
X_train_final = X_train/255
X_test_final = X_test/255
X_val_final = X_val/255

In [ ]:
# Reshape the labels for modeling
y_train = np.reshape(train_labels[:, 0], (5216, 1))
y_test = np.reshape(test_labels[:, 0], (524, 1))
y_val = np.reshape(val_labels[:, 0], (116, 1))

# Check the shape of the transformed labels 
print(np.shape(y_train))
print(np.shape(y_test))
print(np.shape(y_val))

Now that the data is fully manipulated, I begin building my baseline neural network by calling the `Sequential` function, then adding two `Dense` hidden layers with ReLu as my activation functions. I also pass in the input shape to create my input layer. The last layer acts as my output layer with a Sigmoid activation function. 

Once constructed, I compile the baseline model, using parameters I chose from prior research. Therefore, `RMSprop` and binary cross-entropy will act as my optimizer and loss functions, respectively. I preview the model summary in order to highlight the number of parameters being trained in each layer, and the shape of the output. 

In [ ]:
# Create a dense baseline neural network
baseline_model = Sequential()
baseline_model.add(Dense(32, activation='relu', input_shape=(12288, )))
baseline_model.add(Dense(8, activation='relu'))
baseline_model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the baseline model
baseline_model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['acc'])

# List a summary of the baseline model
baseline_model.summary()

There are 393,521 different parameters being trained in this network, but only 1 value will be output. I fit the model to the training data and labels, while also using the validation data to further validate these results. I also set the number of epochs to 50, with a 64 image batch size.

In [ ]:
# Fit the baseline model
baseline = baseline_model.fit(X_train_final, y_train, epochs=50, batch_size=64,
                                      validation_data=(X_val_final, y_val))

In [ ]:
# Evaluate the loss and accuracy scores for the training and validation datasets
print(f'Training data results:\n{baseline_model.evaluate(X_train_final, y_train)}')
print('\n')
print(f'Validation data results:\n{baseline_model.evaluate(X_val_final, y_val)}')

The baseline results above generated a 95.9% accuracy rate for the training data, but only 58.6% for the validation data, which implies the model is overfitting to the training data. To get a better idea, I create a function that will visualize my training results going forward, and apply it to the baseline model.

In [ ]:
# Create a function that will visualize the training results for both datasets
def visualize_training_results(results):
    acc = results.history['acc']
    val_acc = results.history['val_acc']
    loss = results.history['loss']
    val_loss = results.history['val_loss']
    epochs = range(len(acc))
    
    plt.figure()
    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()
    
    plt.figure()
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

visualize_training_results(baseline)

The visuals above further confirm the overfitting theory. The top graph shows how large a gap the accuracies between both datasets really are, which supports our findings. In order to eliminate the overfitting, I must tune the model.

### Regularizing the Baseline Model:
In order to reduce overfitting to the training data, I apply lasso (L2) regularization, specifically with a penalty of 0.01. The network architecture remains similar to the baseline's framework, with the exception of the L2 regularizer addition in the `kernel_regularizer` parameter. I fit the model with the same number of epochs and batch size, and evaluate the model with the validation data. 

In [ ]:
# Regularize the baseline model
regularized_model = Sequential()
regularized_model.add(Dense(32, activation='relu', 
                            kernel_regularizer=regularizers.l2(0.01), input_shape=(12288, )))
regularized_model.add(Dense(8, activation='relu'))
regularized_model.add(Dense(1, activation = 'sigmoid'))
regularized_model.compile(loss='binary_crossentropy', 
                  optimizer='RMSprop', 
                  metrics=['acc'])
results = regularized_model.fit(X_train_final, y_train, epochs=50, batch_size=64,
                                      validation_data=(X_val_final, y_val))

In [ ]:
# Visualize the training results
visualize_training_results(results)

Much better! I see that the data is no longer overfitting on the training data, and even more - the graphs for both datasets follow each other more closely. Albeit far from perfect, the baseline model has improved considerably, and I will cross-validate the results with Scikit-learn's `KFold` technique to calculate the training data's accuracy.

### Cross-Validating the Regularized Baseline Model:

I create a function that will build the regularized baseline model in order to cross-validate on it with `KFold`.

In [ ]:
# Create a function that will build the regularized model above
def build_reg_model():
    regularized_model = Sequential()
    regularized_model.add(Dense(32, activation='relu', 
                            kernel_regularizer=regularizers.l2(0.01), input_shape=(12288, )))
    regularized_model.add(Dense(8, activation='relu'))
    regularized_model.add(Dense(1, activation = 'sigmoid'))
    regularized_model.compile(loss='binary_crossentropy', 
                  optimizer='RMSprop', 
                  metrics=['acc'])
    return regularized_model

I utilize SciKeras, which combines Scikit-learn's machine learning capabilities to further improve Keras's ability to build deep learning models. I'll use `KFold` to cross-validate and produce the average validation scores for both datasets (training and validation).

In [ ]:
# Cross-validate using various scikit-learn tools
keras_model = scikit_learn.KerasClassifier(build_reg_model,
                                          epochs=32,
                                          batch_size=64,
                                          verbose=2)
kfold = KFold(shuffle=True, random_state=seed)
train_scores = cross_val_score(keras_model, X_train_final, y_train, cv=kfold)
validation_scores = cross_val_score(keras_model, X_val_final, y_val, cv=kfold)

In [ ]:
# Calculate the training data's average validation score
print(train_scores.mean())
validation_scores.mean()

Our baseline model has improved greatly - not only did the overfitting issue get solved, but the accuracy of the validation data increased. The model is 74.0% accurate in classifying the validation data, much better than the 58.6% I calculated when I began modeling. This baseline model is a great foundation to work from. 

## Further Modeling
### Convolutional Neural Network (CNN):
The next model I'd like to apply is a Convolutional Neural Network (CNN). The benefits to CNN models are that they're regularized, and are very efficient in analyzing image data. I start with my `Sequential` function, but then apply 3 convolutional layers with two different kernel shapes (`(3, 3)` and `(4, 4)`), and ReLu activation functions. After building my base, I apply `Flatten` and two `Dense` functions to produce a hidden layer before my output layer. I compile and fit the model before visualizing the results.

In [ ]:
# Construct the CNN model's framework
CNNmodel = Sequential()
CNNmodel.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
CNNmodel.add(MaxPooling2D((2, 2)))

CNNmodel.add(Conv2D(32, (4, 4), activation='relu'))
CNNmodel.add(MaxPooling2D((2, 2)))

CNNmodel.add(Conv2D(64, (3, 3), activation='relu'))
CNNmodel.add(MaxPooling2D((2, 2)))

CNNmodel.add(Flatten())
CNNmodel.add(Dense(64, activation='relu'))
CNNmodel.add(Dense(1, activation='sigmoid'))

CNNmodel.compile(loss='binary_crossentropy',
              optimizer="RMSprop",
              metrics=['acc'])

In [ ]:
# Fit the CNN model
CNNresults = CNNmodel.fit(train_images,
                    y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(val_images, y_val))

The plots below 

In [ ]:
# Visualize the training results
visualize_training_results(CNNresults)

### Data Augmentation
In order to improve my model further, I'll increase my training size by applying data augmentation. Considering the images are X-rays, I select my parameters within `ImageDataGenerator` carefully and logically (i.e., increasing brightness to better detect abstract features, zooming in, flipping the image, etc.). I'll reload the training images with these new parameters, as well as the validation and test images (which will also be standardized). 

In [ ]:
# Reshape the training data for augmentation 
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=15, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   brightness_range=[1, 1.5], 
                                   zoom_range=0.2, 
                                   horizontal_flip=True)

In [ ]:
# Get all the data in the training directory (in batches) and standardize
train_generator_aug = train_datagen.flow_from_directory(
    train_data_dir, target_size=(64, 64), 
    batch_size=32, seed=seed, class_mode='binary')

# Get all the data in the test directory and standardize
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_data_dir, target_size=(64, 64), 
    batch_size = 524, seed=seed, class_mode='binary') 

# Get all the data in the validation directory (in batches) and standardize
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    validation_dir, target_size=(64, 64),
    batch_size = 32, seed=seed, class_mode='binary')

Now that the images have been loaded again, I will rebuild my CNN model; however, I will also alter the learning rate in order to improve my CNN model further. 

In [ ]:
# Build a CNN model framework using the augmented training data
augCNNmodel = Sequential()
augCNNmodel.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
augCNNmodel.add(MaxPooling2D((2, 2)))

augCNNmodel.add(Conv2D(32, (4, 4), activation='relu'))
augCNNmodel.add(MaxPooling2D((2, 2)))

augCNNmodel.add(Conv2D(64, (3, 3), activation='relu'))
augCNNmodel.add(MaxPooling2D((2, 2)))

augCNNmodel.add(Flatten())
augCNNmodel.add(Dense(64, activation='relu'))
augCNNmodel.add(Dense(1, activation='sigmoid'))

augCNNmodel.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

I'm also going to create a stopping condition, due to the runtime given the size of the dataset and number of epochs. I'll set that stopping point as a checkpoint with `ModelCheckpoint`, so that I can evaluate the saved model further along the process. Once my conditions have been set, I'll fit the model with these callbacks.

In [ ]:
#Create a file name object for the best model checkpoint
model_filepath = 'best_aug_model.h5'

# Set early stopping and model checkpoint parameters
early_stopping = [EarlyStopping(monitor='val_loss', patience=5), 
                  ModelCheckpoint(filepath=model_filepath, monitor='val_loss', save_best_only=True)]

In [ ]:
# Fit the augmented CNN model
augCNNresults = augCNNmodel.fit(train_generator_aug,
                                epochs=32,
                                callbacks=early_stopping,
                                validation_data=val_generator)

The data-augmented CNN model stopped at 19 epochs, suggesting that is the best point in the model where the validation loss plateaus. The model is saved as `'best_aug_model.h5'`, and I'll load the model in order to evaluate both the training and validation datasets.

In [ ]:
# Load the best (saved) model for the augmented data
saved_model = load_model(model_filepath)

# Calculate the loss and accuracy scores for both training datasets
results_train = saved_model.evaluate(train_generator_aug)
print(f'Training Loss: {results_train[0]:.3} \nTraining Accuracy: {results_train[1]:.3}')

print('----------')

results_val = saved_model.evaluate(val_generator)
print(f'Validation Loss: {results_val[0]:.3} \nValidation Accuracy: {results_val[1]:.3}')

In [ ]:
# Plot the training results
visualize_training_results(augCNNresults)

What an improvement! Not only did the training data show 86% accuracy, the validation data yielded similar results with almost 83% accuracy. The small gap between the two scores isn't surprising, given that CNN models are regularized in nature, and thus are not prone to overfitting. What's more surprising is the growth in accuracy between this model and our baseline, further proving how efficient CNN models are in image classification. 

I also visualize the training results above. One can see the validation metrics are plotted much cleaner, and with less "noise". The two metrics seem to almost mimic each other in their directions, suggesting this is a good model for the task at hand.

### Transfer Learning Approach:
One last approach I want to test out is the transfer learning approach. Transfer learning allows me to utilize pre-trained networks as a base to my model, with the option to build additional layers for specific feature extraction or fine tuning. I researched various pre-trained networks available, and selected `ResNet50` due to its high accuracy and application in medical imaging. 

I print the summary, after instantiating the network, to examine the number of parameters my model will be training - shown to be 23,587,712 total parameters!

In [ ]:
# Instantiate the CNN base object
cnn_base = ResNet50(weights='imagenet', 
                 include_top=False, 
                 input_shape=(64, 64, 3))

# Preview the various parameters that will be trained
cnn_base.summary()

After building the CNN base, I add additional layers (specifically 1 hidden and the output) in order to better analyze the more abstract features within my images. Then, I freeze the base model, and compile my model to fit on the training data (and evaluated by the validation data). I list all the trainable layers and weights as a sanity check to review my work.

In [ ]:
# Build a network with the CNN base
resnet_model = Sequential()
resnet_model.add(cnn_base)
resnet_model.add(Flatten())
resnet_model.add(Dense(256, activation='relu'))
resnet_model.add(Dense(1, activation='sigmoid'))

# Freeze the base model
cnn_base.trainable = False

# List all the trainable layers
for layer in resnet_model.layers:
    print(layer.name, layer.trainable)
    
# Similarly, check how many trainable weights are in the model 
print(f'Trainable weights: {len(resnet_model.trainable_weights)}\n')

# Preview the various parameters and output shapes, once again
resnet_model.summary()

> Note: I only run the model for 10 epochs in order to cut on the training time.

In [ ]:
# Compile the transfer learning model
resnet_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=.001),
              metrics=['acc'])

# Fit the model
resnet_results = resnet_model.fit(train_generator_aug,
                              epochs=10,
                              validation_data=val_generator)

Now that the model has been run, I plot the training results with different markers to better exhibit how the model performed.

In [ ]:
# Plot the training results using different markers
resnet_train_acc = resnet_results.history['acc']
resnet_val_acc = resnet_results.history['val_acc']
resnet_train_loss = resnet_results.history['loss']
resnet_val_loss = resnet_results.history['val_loss']
epch = range(1, len(resnet_train_acc) + 1)

plt.plot(epch, resnet_train_acc, 'r.', label='Training Accuracy')
plt.plot(epch, resnet_val_acc, 'b', label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()
plt.figure()
plt.plot(epch, resnet_train_loss, 'r.', label='Training Loss')
plt.plot(epch, resnet_val_loss, 'b', label='Validation Loss')
plt.title('Loss')
plt.legend()
plt.show()

........... (EVALUATE TRANSFER)

In [ ]:
# Calculate the loss and accuracy scores for both training datasets
results_train = saved_model.evaluate(train_generator_aug)
print(f'Training Loss: {results_train[0]:.3} \nTraining Accuracy: {results_train[1]:.3}')

print('----------')

results_val = saved_model.evaluate(val_generator)
print(f'Validation Loss: {results_val[0]:.3} \nValidation Accuracy: {results_val[1]:.3}')

## Deployment & Evaluation
After reviewing the training and validation scores, I've determined the CNN model with the augmented data is the best at classifying my X-ray images. Fortunately, this model was saved by `ModelCheckpoint`, and loaded earlier in the notebook; therefore, evaluating the testing dataset is simple.

In [ ]:
# Evaluate the test data and list testing accuracy
test_loss, test_acc = saved_model.evaluate(test_generator)
y_hat_test = saved_model.predict(test_generator)
print('Generated {} predictions'.format(len(y_hat_test)))
print('test acc:', test_acc)

My saved model is 82.3% accurate after analyzing the testing data, which is decent but obviously leaves room for improvement. Below I plot the confusion matrix to get a further look at how the model performed. 

In [ ]:
y_test = test_generator.classes
y_hat_test[y_hat_test > 0.5] = 1
y_hat_test[y_hat_test < 0.5] = 0
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix(y_test, y_hat_test), annot=True,
           fmt='.3g', xticklabels=['Normal', 'Pneumonia'],
           yticklabels=['Normal', 'Pneumonia'], cmap='OrRd', cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

The visual above shows this model still has room for improvement, especially in its false negative count (predicting Normal when the patient actually has pneumonia!).

## Conclusion

This analysis leads to the following conclusions:

The Support Vector Machine (SVM) model served as our best algorithm for identifying pop songs and classifying them correctly.

The most important features for identifying a pop track is its speechiness, danceability, acousticness & popularity. The first two features have strong correlations with the target, and the other half exhibit similar importance to the model in its classification task.

## Limitations & Further Work

This project is limited in a few ways, the first being it only takes into account the top 100 songs from the last 23 years. In addition, the genres in the original dataset varied and listed many subgenres, which complicated the analysis. Lastly, the range of hyperparameters selected for our parameter grids could have been selected better; however, this would have been computationally heavy. This limited the range we could include and analyze, leaving much to be desired.

Further analyses could yield additional insights to which features are important to an effective classifier, and possibly improve our algorithm. Some possible courses of action we could take include dimensionality reduction, in order to reduce the noise in the dataset that may be causing our low precision scores. In addition, we could consolidate the other genres in the dataset in order to make classification easier; however, this would be an unrealistic (or "ideal") analysis due to the subjective nature behind music genres. we could also feed more data and track information in order to train our models to better identify other genres and continue to improve our precision.